# Embedding Similarity & Weight Projection - M2V

After extracting the learned node embeddings from the LastFM database using Metapath2Vec, we will input and process the respective CSV and txt files to calculate `Cosine Similarity` between any two nodes sharing an edge in the original graph.

We first import the required libraries.

In [12]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

## Loading Embeddings Data from CSV

Since your embeddings are saved in a CSV file, we will use Pandas to load this file into a DataFrame. Each row in CSV file represents a node, and each column represents a feature of the embeddings (i.e., 128-dimension embeddings).

In [13]:
embeddings_df = pd.read_csv('M2V_Embeddings/node_embeddings.csv', delimiter=',', header=None, float_precision='high')

with open('M2V_Embeddings/node_ids.txt', 'r') as file:
    node_indexes = [line.strip() for line in file]

#Remove formatting of _ in M2V List
node_indexes = [node_id.replace('_', '') for node_id in node_indexes]

# Add node_indexes back as the first column of the DataFrame
embeddings_df.insert(0, 'node_id', node_indexes)

# Set node indexes as embeddings_df index to allow for faster search later on
embeddings_df.set_index('node_id', inplace=True)

# Now 'embeddings_df' is ready for further analysis
print(embeddings_df.head())

embeddings_df.shape

                 0         1         2         3         4         5    \
node_id                                                                  
co3         0.564233 -0.163861  0.570580  0.038262  0.079470 -0.282460   
u174194590  0.410928  0.096629 -0.128370 -0.495595 -0.130103  0.200706   
ci17       -0.115506 -0.514996 -0.540942  0.070929  0.719329 -0.335045   
ci5         0.544065 -0.760291  0.686224 -0.300136  0.185406 -0.837266   
co4        -0.094877 -0.615090  0.048740 -0.797129 -0.075828 -0.290816   

                 6         7         8         9    ...       118       119  \
node_id                                             ...                       
co3         0.310749 -0.555545  0.039620 -0.361103  ... -0.348461  0.155472   
u174194590 -0.058844  0.061227  0.273708 -0.064791  ... -0.415627  0.174562   
ci17        0.469154 -0.120295 -0.620274  0.525111  ... -0.129488  0.096290   
ci5        -0.702685 -0.455983  0.591897 -0.375288  ... -0.440158 -0.084325   
co4    

(245621, 128)

Now that we have cleaned-up the embeddings into a dataframe, we need to check if there are any inconsistencies in the data. We also check for non-numeric data.

In [14]:
# Check for non-numeric data
print("Data types:\n", embeddings_df.dtypes)

# Check for missing values
if embeddings_df.isnull().values.any():
    print("Missing values found")

# Check shape of embeddings dataframe to see if there are varying row lengths
print("DataFrame shape:", embeddings_df.shape)


Data types:
 0      float64
1      float64
2      float64
3      float64
4      float64
        ...   
123    float64
124    float64
125    float64
126    float64
127    float64
Length: 128, dtype: object
DataFrame shape: (245621, 128)


## Loading Edge List Data from .edgelist File

To be able to access which nodes are connected by an edge, we need to import the edge list into another dataframe. Note that the node IDs must be consistent across both the embedding and edge list dataframes! It is also an undirected graph, meaning source and target do not necessarily mean anything.

In [17]:
# File path
edgelist_file = 'EdgeList_MusicMicro/musicmicro.edgelist'

# Read edge list into DataFrame
edge_list_df = pd.read_csv(edgelist_file, sep=' ', header=None, names=['source', 'target'])

display(edge_list_df)

display(embeddings_df.head())

,source,target
0,u74717431,t7748381
1,u127821914,t3529910
2,u174194590,t5762915
3,u141847381,t6987845
4,u87215499,t4082536
...,...,...
641279,ci20717,co3
641280,ci20718,co9
641281,ci20719,co5
641282,ci20720,co3


,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
node_id,,,,,,,,,,,,,,,,,,,,,
co3,0.564233,-0.163861,0.570580,0.038262,0.079470,-0.282460,0.310749,-0.555545,0.039620,-0.361103,...,-0.348461,0.155472,0.457085,0.641228,0.136438,-0.232058,0.226928,0.562846,-0.270781,0.806977
u174194590,0.410928,0.096629,-0.128370,-0.495595,-0.130103,0.200706,-0.058844,0.061227,0.273708,-0.064791,...,-0.415627,0.174562,0.044197,-0.224626,-0.028417,0.355082,-0.340714,-0.527494,-0.052179,0.071709
ci17,-0.115506,-0.514996,-0.540942,0.070929,0.719329,-0.335045,0.469154,-0.120295,-0.620274,0.525111,...,-0.129488,0.096290,0.038152,0.109173,-0.866303,0.533571,-0.328243,0.337223,0.165756,0.353211
ci5,0.544065,-0.760291,0.686224,-0.300136,0.185406,-0.837266,-0.702685,-0.455983,0.591897,-0.375288,...,-0.440158,-0.084325,-0.720733,0.167153,-0.311992,0.063178,-0.385549,-0.086492,-0.290417,0.048780
co4,-0.094877,-0.615090,0.048740,-0.797129,-0.075828,-0.290816,0.009609,-0.210236,-0.374844,-0.364485,...,-0.223284,0.859846,-0.240655,0.906691,-0.085079,0.356791,-1.468777,-0.248339,-0.339965,0.792251


#### **IMPORTANT: M2V did not cover all the nodes in the original graph, meaning that there will be edges in the edge list that will NOT be considered!**

In [18]:
# Assume edge_list_df is your DataFrame containing the edges

# Filter edges where both the source and target nodes exist in node_indexes
filtered_edge_list_df = edge_list_df[edge_list_df['source'].isin(node_indexes) & edge_list_df['target'].isin(node_indexes)]

# filtered_edge_list_df now contains only edges where both nodes are present in embeddings_df
display(filtered_edge_list_df)

,source,target
0,u74717431,t7748381
1,u127821914,t3529910
2,u174194590,t5762915
3,u141847381,t6987845
4,u87215499,t4082536
...,...,...
641279,ci20717,co3
641280,ci20718,co9
641281,ci20719,co5
641282,ci20720,co3


## Calculating Cosine Similarity

- For each edge, we retrieve the embeddings of the connected nodes.
- Use cosine_similarity from sklearn.metrics.pairwise to calculate the similarity for each edge.
- Store the similarity values in a new column in the edge list DataFrame.

### Method 1: Row-by-Row Iteration (Slower, Inefficient) --> SKIP

For graphs with a very large number of edges, iterating over each row using DataFrame.iterrows() and calculating cosine similarity one pair at a time can be very inefficient. This method has a time complexity that grows linearly with the number of edges, leading to long execution times for large graphs. 

In [ ]:
# Assume embeddings_df is your DataFrame with embeddings indexed by node IDs
# Calculate cosine similarities
similarities = []
for _, row in edge_list_df.iterrows():
    emb1 = embeddings_df.loc[row['source']].values.reshape(1, -1)
    emb2 = embeddings_df.loc[row['target']].values.reshape(1, -1)
    similarity = cosine_similarity(emb1, emb2)[0, 0]
    similarities.append(similarity)

# Add similarities to the edge list DataFrame
edge_list_df['weight'] = similarities

### Method 2: Batch Processing using Vectorization (Faster, Efficient)

1. Efficiency and Vectorization
    - Vectorized Operations: Modern CPUs and computing frameworks like NumPy are optimized for vectorized operations, where the same operation is performed simultaneously on multiple data points. This is inherently more efficient than processing each data point (or in this case, each pair of embeddings) individually, as it minimizes the overhead associated with looping constructs in high-level languages like Python.

    - Batch Processing: By processing multiple pairs of embeddings at once, the batch approach reduces the number of iterations and takes full advantage of vectorized operations. This leads to a significant reduction in computation time, especially for large datasets.

2. Scalability
    - Memory Management: Calculating cosine similarities for millions of edges at once can be memory-intensive, leading to memory overflow or significantly slowed performance due to swapping. Processing the data in smaller batches helps manage memory usage more effectively, ensuring that the computation remains within the available system resources, thereby maintaining performance across varying scales of data.

    - Parallelization Potential: Although not implemented in the provided code, batch processing opens up possibilities for parallel computation. Batches can be processed in parallel across multiple CPU cores or even distributed systems, further speeding up the computation for very large graphs.

3. Practicality
    - Adaptability: The batch size can be adjusted based on the available computing resources and the specific requirements of the dataset. This flexibility allows the method to be optimized for different environments, from personal laptops to high-performance computing clusters.

    - Reduced Computational Overhead: The original method's reliance on DataFrame.iterrows() is known to be inefficient for large datasets due to the overhead of generating Series objects for each row. In contrast, the batch processing approach minimizes this overhead by working directly with NumPy arrays, which are more efficient both in terms of memory layout and computational performance.

The similarities list is initialized with NaN values for the length of edge_list_df. This assumes all edges initially don't have similarities calculated.

For each batch, it creates a list of valid edges (batch_edges) and their corresponding indices (batch_indices) in the original edge_list_df.
It then calculates the similarities only for these valid edges.

The computed similarities are assigned back to their respective positions in the similarities list based on batch_indices.

In [19]:
# Assume embeddings_df is indexed by node IDs and contains embeddings
embeddings = embeddings_df.to_numpy()

# Map node IDs to their index in the embeddings array for quick lookup
node_id_to_index = {node_id: index for index, node_id in enumerate(embeddings_df.index)}

# Convert filtered edge list source and target to indices
edge_indices = [(node_id_to_index[row['source']], node_id_to_index[row['target']])
                for _, row in filtered_edge_list_df.iterrows()]

# Calculate similarities in batches to manage memory usage
batch_size = 1000  # Adjust based on your memory capacity
similarities = []

for i in range(0, len(edge_indices), batch_size):
    batch_edges = edge_indices[i:i+batch_size]
    emb1 = np.array([embeddings[index_pair[0]] for index_pair in batch_edges])
    emb2 = np.array([embeddings[index_pair[1]] for index_pair in batch_edges])
    
    # Calculate batch similarities
    batch_similarities = cosine_similarity(emb1, emb2).diagonal()
    similarities.extend(batch_similarities)

# Add similarities to the filtered edge list DataFrame
filtered_edge_list_df['weight'] = similarities


/Users/kareemrahman/.pyenv/versions/3.6.15/lib/python3.6/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [20]:
display(filtered_edge_list_df)

,source,target,weight
0,u74717431,t7748381,0.892881
1,u127821914,t3529910,0.717761
2,u174194590,t5762915,0.483259
3,u141847381,t6987845,0.720264
4,u87215499,t4082536,0.754541
...,...,...,...
641279,ci20717,co3,0.772626
641280,ci20718,co9,0.878173
641281,ci20719,co5,0.737660
641282,ci20720,co3,0.751880


We can now export the new updated edge list with cosine similarities as edge weights.

In [21]:
# Optionally save the updated edge list
filtered_edge_list_df.to_csv('M2V_edge_list_with_similarity.csv', index=False)

## Projecting Weights to New Homogeneous Graph

- A new graph that mimics the same structure as its original heterogeneous counterpart, but ignores node types and edge types. This information should already be embedded structurally and semantically in the node embeddings.
- Based on the cosine similarity calculations, the values are projected onto the graph as edge weights.
- This graph will be constructed using StellarGraph (can be later converted into NetworkX or Adjacency Matrices + Edge Lists based on CD algorithm) 

In [22]:
from stellargraph import StellarGraph

/Users/kareemrahman/.pyenv/versions/3.6.15/lib/python3.6/site-packages/requests/__init__.py:104: RequestsDependencyWarning: urllib3 (1.26.18) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  RequestsDependencyWarning)


In [23]:

# Assuming edge_list_df columns are ['source', 'target', 'weight']

# Create StellarGraph from edge list with weights
G = StellarGraph(edges=filtered_edge_list_df)

print(
    "Number of nodes {} and number of edges {} in graph.".format(
        G.number_of_nodes(), G.number_of_edges()
    )
)

print("\n")

print("Below is an overview of the StellarGraph structure:")
print(G.info())

Number of nodes 245621 and number of edges 641143 in graph.


Below is an overview of the StellarGraph structure:
StellarGraph: Undirected multigraph
 Nodes: 245621, Edges: 641143

 Node types:
  default: [245621]
    Features: none
    Edge types: default-default->default

 Edge types:
    default-default->default: [641143]
        Weights: range=[0.142078, 0.999431], mean=0.692594, std=0.138812
        Features: none
